In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import pandas as pd
import seaborn as sns
#from scipy import stats

from func_utils import *

#Two lines to make project autoimport again after 
#every modifications in func_utils
%load_ext autoreload
%autoreload 2

# File reading

In [ ]:
from func_utils import getDataFilePath
df = pd.read_excel(getDataFilePath(), sheet_name = 'Tabella')

# Data initial filtering



In [ ]:
valid_claim_values = ['ZW01','ZW02','ZW06']
valid_part_number_descr_values = ['-ECONOMY', '-ECONOMY50', '-E1', '-E3']

#filter by claim_type and part_number_des
filt_df = filterByColumnValues(df,'claim_type',valid_claim_values)
filt_df = filterByColumnValues(filt_df,'part_number_des',valid_part_number_descr_values)

#filt_df

# Separate df by country

In [ ]:
countries_list=getTableColumn(filt_df,'country',True)#Get a list with all the present countries

df_list = separateDfByCountry(filt_df,countries_list)

## Filtering of data only for the most 30 common components 

In [ ]:
numOfComponentToAnalyze= 30 #Number of most common components to analyze
mostCommonComponents = [] 
componentsFrequencies = Counter(filt_df['component']).most_common(numOfComponentToAnalyze) #list of couples comp-frequency

for component in componentsFrequencies : 
    mostCommonComponents.append(component[0])

In [ ]:
df_list_comp_filt = [] #df_list but filtering only the most common components

for countryTable in df_list:
    df_list_comp_filt.append(filterByColumnValues(countryTable,'component',mostCommonComponents))

len(df_list_comp_filt[2])    

## Additional filter excluding the rows that have working hours or amount equal to zero

In [ ]:
df_list_without_outliers=[] #Removing data with 0 working hours and 0 net gain
for countryTable in df_list_comp_filt:
    df_list_without_outliers.append(countryTable[(countryTable['work_hours']>0) & (countryTable['amount']>0)])

## Dataframe with outliers

In [ ]:
df_list_outliers=[] #Data with 0 working hours and 0 net gain
for countryTable in df_list_comp_filt:
    df_list_outliers.append(countryTable[(countryTable['work_hours']==0) | (countryTable['amount']==0)])

print('Outliers number:', len(df_list_outliers[1]))    

## Plot with outliers

In [ ]:
currentNation = 'Austria'

countryIndex = getValueIndex(countries_list,currentNation)

g=sns.relplot(
    data=df_list_comp_filt[countryIndex],
    x="work_hours", y="amount",
    hue="product_division"
)
g.fig.suptitle(currentNation)

## Plot without outliers

In [ ]:
currentNation = 'Italy'

countryIndex = getValueIndex(countries_list,currentNation)

g=sns.relplot(
    data=df_list_without_outliers[countryIndex],
    x="work_hours", y="amount",
    hue="component"
)
g.fig.suptitle(currentNation)

## Adding to the no-outliers dataframe another column containing the ratio amount/work_hours

In [ ]:
ratio_df_list_without_outliers = df_list_without_outliers  #new df that will contain the new column

for countryTable in ratio_df_list_without_outliers:
    countryTable['amount/work_hours'] = countryTable['amount']/countryTable['work_hours']

ratio_df_list_without_outliers[1]

## Boxplot for countries

In [ ]:
currentNation = 'Germany'
countryIndex = getValueIndex(countries_list,currentNation)

sns.boxplot(x=ratio_df_list_without_outliers[countryIndex]["amount/work_hours"]
,y=ratio_df_list_without_outliers[countryIndex]["component"])

plt.title(currentNation)

## Addin a column containing the z score of each amount/work hours ratio (for each component)

In [ ]:
zscore_df_list_without_outliers = ratio_df_list_without_outliers  #new df that will contain the new column

for countryTable in zscore_df_list_without_outliers:
    countryTable['z_score'] = computeZScoreColumn(countryTable,'component','amount/work_hours')

#zscore_df_list_without_outliers[1]['z_score'].tolist()

## I filter out the rows with z_score (absolute value) bigger than 3

In [ ]:
df_filt_zscore=[] #List with values of the previous df, removing the rows with abs(zscore)>3

for countryTable in zscore_df_list_without_outliers:
    df_filt_zscore.append(countryTable[(abs(countryTable['z_score'])<=3)])


#print(getTableColumn(df_filt_zscore[1],'z_score', False))

## Dataframe containing only the z_score outliers removed in the previous cell

In [ ]:
df_zscore_outliers = [] #List containing the z_score outliers 

for countryTable in zscore_df_list_without_outliers:
    df_zscore_outliers.append(countryTable[(abs(countryTable['z_score'])>3)])

#print(getTableColumn(df_zscore_outliers[1],'z_score', False))

## Outliers percentage per country

In [71]:
outliers_percentages = []

for i in range( len(countries_list)):
    outliers_percentages.append((len(df_list_outliers[i].index)+len(df_zscore_outliers[i].index))/len(df_list[i].index)) 

#print(outliers_percentages)       

[0.008166078956438741, 0.01023297664940587, 0.014307917352349608, 0.0020833333333333333, 0.01866238253482691, 0.012424199016377787, 0.012656935796672451, 0.009207323526299079, 0.025528520143597924, 0.010617760617760617, 0.021216407355021217, 0.015978054238756405, 0.008095199546668826, 0.025627615062761507, 0.013981569748967271, 0.03655913978494624, 0.011050794881737108, 0.010529373320746497, 0.03145853193517636, 0.0044994375703037125, 0.007877664504170528, 0.015703869882220976, 0.0, 0.024390243902439025, 0.008888888888888889, 0.043478260869565216, 0.007575757575757576, 0.010471204188481676, 0.00936768149882904, 0.018072289156626505, 0.11428571428571428, 0.0, 1.0, 0.0, 0.0]
